## Part 1: Pulse Rate Algorithm

### Dataset
 **Troika** dataset is used to build the algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data.
1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

- **Code Description** - The code is used to test the performance of the algorithm on the training dataset. You can run the `Evaluate` function to start the algorithm.

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import glob
import scipy.stats
import scipy.signal
import os

### Loading Dataset

- **Data Description**
    - The dataset consist of signals from accelerometer and PPG sensors. 
    - The accelerometer has three channels, each corresponding to a space axis x, y, and z. I use the magnitude of these three channels as a distance calculation. 
    - Both signals have noise but combining them would produce good results. 
    - The data is sampled at 125Hz from 12 different subjects. 

In [2]:
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

In [3]:
def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

In [4]:
data_fls, ref_fls = LoadTroikaDataset()

In [5]:
len(data_fls), len(ref_fls)

(12, 12)

In [6]:
ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

In [7]:
reference_signal = sp.io.loadmat(ref_fls[0])

In [8]:
# heartbeat reference data
"""
Variable 'BPM0', which gives the BPM value in every 8-second time window. Note that two successive time windows 
overlap by 6 seconds. Thus the first value in 'BPM0' gives the calcualted heart rate ground-truth in 
the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate ground-truth 
from the 3rd second to the 10th second. 

"""

reference_signal['BPM0'].reshape(-1), len(reference_signal['BPM0'])

(array([ 74.33920705,  76.35746606,  77.14285714,  74.66814159,
         72.58064516,  71.68458781,  72.89416847,  73.44940152,
         75.33482143,  76.8442623 ,  79.5990566 ,  79.11392405,
         74.50331126,  70.83825266,  69.58762887,  71.94244604,
         77.31958763,  80.29978587,  82.87292818,  83.51893096,
         84.65011287,  88.23529412,  90.95920617,  92.87925697,
         94.73684211,  97.28773585,  99.66777409, 101.58013544,
        102.84810127, 103.44827586, 103.17460317, 102.95670539,
        103.28389831, 104.05549626, 106.20915033, 108.58324716,
        110.66969353, 111.58342189, 112.29946524, 112.78195489,
        113.63636364, 114.50381679, 115.13157895, 116.02209945,
        117.05685619, 118.54583772, 120.19230769, 121.8851571 ,
        123.96694215, 125.78616352, 128.06830309, 130.86150491,
        133.18534961, 135.49415515, 137.19512195, 138.88888889,
        140.57331863, 141.65792235, 142.70613108, 143.00847458,
        143.31210191, 143.46439957, 143.

In [9]:
#The first row has the ppg value
#The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis).
ppg, accx, accy, accz

(array([  4. ,   6. ,   3. , ...,  86. , 104. , 118.5]),
 array([-0.0702, -0.0702, -0.0546, ...,  0.4134,  0.4134,  0.4134]),
 array([ 0.3432,  0.3588,  0.3666, ..., -0.2808, -0.273 , -0.273 ]),
 array([0.9594, 0.9438, 0.936 , ..., 0.7254, 0.7176, 0.7254]))

In [10]:
len(ppg), len(accx), len(accy), len(accz)

(37937, 37937, 37937, 37937)

### Defining Error Metric Function

In [11]:
def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)
    print("The best 90% confidence estimates", percentile90_confidence)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

## Pulse Rate Algorithm

- **Algorithm Description** 
    1. The algorithm works as follows:  
        - Performs bandpass filtering on both signals.
        - Create features that would be helpful for the algorithm
        - Train a RandomForestRegression.
        
    2. The specific aspects of the physiology that it takes advantage of: 
        - I used PPG signals for measuring heart rate. 
        - The capillaries in the wrist are filled with blood when the ventricles contract.
        - When there are few red blood cells in these capillaries, the light from the PPG sensor will be reflected in a large amount. 
        - When the blood cells are many, they absorb the light, so the optical detector will detect less light.
        - Changes in the light detected by the optical detector, will produce an oscillating waveform which is the pulse rate.
        
    3. A description of the algorithm outputs: The algorithm outputs the estimated frequency (in BPM) along with its confidence score.
    
    4. Confidence on algorithm outputs: The confidence is calculated based on the magnitude of a small area that contains the estimated spectral frequency relative to the magnitude sum of the entire spectrum.
    
    5. Common failure modes: 
        - The algorithm may not perform well in other stages of changing heart rate due to arm and hand movements.
        - The length of records in data files are not equal.

In [12]:
def RunPulseRateAlgorithm(data_fl, ref_fl):
    """ Calculates mean absolute errors and confidences
    
    Args:
        data_fl: (string) Path to the signal file
        ref_fl: (string) Path to the reference signal file
        
    Returns:
        (np.array) Mean absolute errors
        (np.array) Confidences
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # load the reference signal
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    errs = []
    confs = []
    
    start_indices, end_indices = get_start_end(len(accx), len(ground_truth))
    
    for i, start in enumerate(start_indices):
        end = end_indices[i]
        ref = ground_truth[i]
        
        # Bandpass filtering the signals
        w_ppg =  bandpass_filter(ppg[start:end])
        w_accx = bandpass_filter(accx[start:end])
        w_accy = bandpass_filter(accy[start:end])
        w_accz = bandpass_filter(accz[start:end])
        
        # Get features
        feature = Featurize(w_ppg, w_accx, w_accy, w_accz)
        
        # Get prediction
        pred = model.predict(np.reshape(feature, (1, -1)))[0]
        
        # calculate confidence
        n = len(w_ppg) * 4
        freqs = np.fft.rfftfreq(n, 1/fs)
        fft = np.abs(np.fft.rfft(w_ppg,n))
        fft[freqs <= 40/60.0] = 0.0
        fft[freqs >= 240/60.0] = 0.0
    
        est_fs = pred / 55.0
        fs_win = 30  / 60.0
        fs_win_e = (freqs >= est_fs - fs_win) & (freqs <= est_fs +fs_win)
        conf = np.sum(fft[fs_win_e])/np.sum(fft)
        
        errs.append(np.abs(pred - ref))
        confs.append(conf)
    
    errs = np.array(errs)
    confs = np.array(confs)
    # Return mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errs, confs

In [13]:
fs = 125
window_shift = 2
window_length = 12

In [14]:
def get_start_end(sig_len, ref_len):
    """ Returns the start and end indices of a signal """
    n = ref_len if ref_len < sig_len else sig_len
    start = (np.cumsum(np.ones(n) * fs * window_shift) - fs * window_shift).astype(int) # windows shift of 2
    return (start, start + window_length * fs) # windows length of 12

In [15]:
def bandpass_filter(signal, band_pass = (40/60, 240/60), fs = fs):
    """ Performs a bandpass filter on the signal. """
    
    b,a = scipy.signal.butter(3, band_pass, fs=fs, btype= 'bandpass')
    
    # Perform forward and backward digital butterworth filter
    return scipy.signal.filtfilt(b,a,signal)

In [16]:
def Featurize(ppg, accx, accy, accz):
    """ Create features """

    n = len(ppg) * 4
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft = np.abs(np.fft.rfft(ppg,n))
    fft[freqs <= 40/60.0] = 0.0
    fft[freqs >= 240/60.0] = 0.0
    
    acc_mag = np.sqrt(accx**2 + accy**2 + accz**2)
    acc_fft = np.abs(np.fft.rfft(acc_mag, n))
    acc_fft[freqs <= 40/60.0] = 0.0
    acc_fft[freqs >= 240/60.0] = 0.0
    
    ppg_feature = freqs[np.argmax(fft)]
    acc_feature = freqs[np.argmax(acc_fft)]
    
    return np.array([ppg_feature, acc_feature])

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

model = None

In [18]:
def train_model():
    global model
    
    if model: return
    
    print("Starting...")
    data_fls, ref_fls = LoadTroikaDataset()
    
    targets, features, subjects = [], [], []
    
    for data_fl, ref_fl in (zip(data_fls, ref_fls)):
        
        # Load data using LoadTroikaDataFile
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)

        # load the reference signal
        ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)

        start_indices, end_indices = get_start_end(len(accx), len(ground_truth))
        
        subject_name = os.path.basename(data_fl).split('.')[0]  
        for i, start in enumerate(start_indices):
            end = end_indices[i]
            
            # Bandpass filtering the signals
            w_ppg =  bandpass_filter(ppg[start:end])
            w_accx = bandpass_filter(accx[start:end])
            w_accy = bandpass_filter(accy[start:end])
            w_accz = bandpass_filter(accz[start:end])

            # Get features
            feature = Featurize(w_ppg, w_accx, w_accy, w_accz)

            targets.append(ground_truth[i])
            features.append(feature)
            subjects.append(subject_name)
            
    targets = np.array(targets)
    features = np.array(features)
    
    # Start training
    regression = RandomForestRegressor(n_estimators=220,max_depth=15)
    lf = KFold(n_splits=8)
    splits = lf.split(features,targets,subjects)

    for i, (train_idx, test_idx) in enumerate(splits):
        X_train, y_train = features[train_idx], targets[train_idx]
        X_test, y_test = features[test_idx], targets[test_idx]
        regression.fit(X_train, y_train)
    
    model = regression

In [19]:
def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Train the model
    train_model()
    
    print("Start evaluating...")
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

In [20]:
Evaluate()

Starting...
Start evaluating...
The best 90% confidence estimates 0.3904913152188008


7.81324105520044

- **Algorithm Performance**
    - The performance is calculated using the mean absolute error between the estimations and the ground truth signal.
    - To limit the noise coming from hand and arm movements, I used the acceleration magnitude. Also, the accelerometer magnitude is close to 1 g, which means the total force on the accelerometer is only due to gravity. But if you look at the individual channels, they shift around.
    - One can use other metrics like accuracy, recall, percision, f1-score.
    - The mean absolute error of the algorithm when evaluated on the test set is 7.81.
        